In [4]:
import networkx as nx
import matplotlib.pyplot as plt
import scipy as sci
import numpy as np
import numpy.linalg as linalg
import numpy.random as rand
import pandas as pd
from platform import node
from cdlib.classes.node_clustering import NodeClustering
import community.community_louvain
import cdlib as cd
from collections import defaultdict
from collections import Counter
import networkx as nx
import cdlib.algorithms as algo
import cdlib.viz as viz
import matplotlib.pyplot as plt
import pyvis
from pyvis.network import Network
import random

from functions import louvain

In [5]:
def get_degrees(edges):
    counter = Counter(edges)
    return counter.most_common()

In [6]:
# reading in the data, creating the graph

hero_edges = pd.read_csv('hero-network.csv')
nodes = pd.read_csv('nodes.csv')
edges = pd.read_csv('edges.csv')

hero_edges['hero1'].str.strip()
hero_edges['hero2'].str.strip()

hero_graph = nx.from_pandas_edgelist(hero_edges, 'hero1', 'hero2', create_using=nx.MultiGraph())

In [7]:
list(hero_graph.edges)[400]

("BLACK PANTHER/T'CHAL", 'CAPTAIN AMERICA', 26)

In [8]:
# get degrees of each node

hero_nodes = []
for x in range(len(nodes['node'])):
    if nodes['type'][x] != 'comic':
        hero_nodes.append(nodes['node'][x])

stubs1 = list(hero_edges['hero1'])
stubs2 = list(hero_edges['hero2'])

stubs = stubs1 + stubs2

hero_degrees = get_degrees(stubs)
degrees = [x[1] for x in hero_degrees]
heroes_sorted = [x[0] for x in hero_degrees]

In [9]:
MarvelCluster = louvain(hero_graph)

In [10]:
print(len(MarvelCluster.communities))
for i in range(len(MarvelCluster.communities)):
    MarvelCluster.communities[i] = sorted(MarvelCluster.communities[i], key = heroes_sorted.index)

18


In [11]:
coms = pd.DataFrame(MarvelCluster.communities)
coms = coms.transpose()

In [12]:
coms_mod = nx.community.modularity(hero_graph, list(MarvelCluster.communities))
print(coms_mod)

0.47164642659544814


In [14]:
coms.head(50)
coms.to_csv('communities.csv', index=False)

In [16]:
coms_map = dict()
for sub in MarvelCluster.communities:
    for node in sub:
        coms_map[node] = sub[0]

print(coms_map)


graph_contracted = community_louvain.induced_graph(coms_map, hero_graph)
graph_contracted.remove_edges_from(nx.selfloop_edges(graph_contracted))

{'WOLVERINE/LOGAN ': 'WOLVERINE/LOGAN ', 'BEAST/HENRY &HANK& P': 'WOLVERINE/LOGAN ', 'CYCLOPS/SCOTT SUMMER': 'WOLVERINE/LOGAN ', 'STORM/ORORO MUNROE S': 'WOLVERINE/LOGAN ', 'COLOSSUS II/PETER RA': 'WOLVERINE/LOGAN ', 'PROFESSOR X/CHARLES ': 'WOLVERINE/LOGAN ', 'MARVEL GIRL/JEAN GRE': 'WOLVERINE/LOGAN ', 'ANGEL/WARREN KENNETH': 'WOLVERINE/LOGAN ', 'ICEMAN/ROBERT BOBBY ': 'WOLVERINE/LOGAN ', 'NIGHTCRAWLER/KURT WA': 'WOLVERINE/LOGAN ', 'ROGUE /': 'WOLVERINE/LOGAN ', 'SHADOWCAT/KATHERINE ': 'WOLVERINE/LOGAN ', 'CANNONBALL II/SAM GU': 'WOLVERINE/LOGAN ', 'PSYLOCKE/ELISABETH B': 'WOLVERINE/LOGAN ', 'WOLFSBANE/RAHNE SINC': 'WOLVERINE/LOGAN ', 'GAMBIT/REMY LEBEAU ': 'WOLVERINE/LOGAN ', 'MAGNETO/MAGNUS/ERIC ': 'WOLVERINE/LOGAN ', 'POLARIS/LORNA DANE': 'WOLVERINE/LOGAN ', 'SUMMERS, NATHAN CHRI': 'WOLVERINE/LOGAN ', 'SUNSPOT/ROBERTO DACO': 'WOLVERINE/LOGAN ', 'BANSHEE/SEAN CASSIDY': 'WOLVERINE/LOGAN ', 'BOOMER/TABITHA SMITH': 'WOLVERINE/LOGAN ', 'MACTAGGERT, MOIRA KI': 'WOLVERINE/LOGAN ', 'JUBILE

In [17]:
#centers = [x[0] for x in coms]
centers = list(graph_contracted.nodes)
min_list = []

for x in centers:
    #min_list.append(list(graph_contracted.edges)[x][y])
    for y in list(hero_graph.edges):
        if x in y:
            min_list.append(y)



In [ ]:
min_list[10]

("BLACK PANTHER/T'CHAL", 'WOLVERINE/LOGAN ', 7)

In [18]:
subset_graph = nx.MultiGraph()
subset_graph.add_weighted_edges_from(min_list)

In [19]:
subset_louvain = louvain(subset_graph, weight = 'weight')

In [26]:
subset_map = dict()
for sub in subset_louvain.communities:
    for node in sub:
        subset_map[node] = sub[0]


subset_contracted = community_louvain.induced_graph(subset_map, subset_graph)
subset_contracted.remove_edges_from(nx.selfloop_edges(subset_contracted))

In [29]:
coms.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,WOLVERINE/LOGAN,CAPTAIN AMERICA,SPIDER-MAN/PETER PAR,HULK/DR. ROBERT BRUC,THING/BENJAMIN J. GR,SILVER SURFER/NORRIN,NOVA/RICHARD RIDER,THOR/DR. DONALD BLAK,QUASAR III/WENDELL V,PATRIOT/JEFF MACE,"FURY, COL. NICHOLAS",HAVOK/ALEX SUMMERS,"MASON, LOUISE",CHI-CHEE,"LUDLUM, ROSS",PANTHER CUB/,DR. MIDAS/,STEEL SPIDER/OLLIE O
1,BEAST/HENRY &HANK& P,IRON MAN/TONY STARK,"JAMESON, J. JONAH",DR. STRANGE/STEPHEN,HUMAN TORCH/JOHNNY S,MOONDRAGON/HEATHER D,FIRESTAR/ANGELICA JO,ODIN [ASGARDIAN],SERSI/SYLVIA,MISS AMERICA/MADELIN,"DUGAN, TIMOTHY ALOYI",BLOODSTORM | MUTANT,"ARCHER, ULYSSES SAMS",HOPE,ORWELL,STERLING,MARVEL BOY VIII/NOH-,MASTER OF VENGEANCE
2,CYCLOPS/SCOTT SUMMER,SCARLET WITCH/WANDA,"WATSON-PARKER, MARY","JONES, RICHARD MILHO",MR. FANTASTIC/REED R,SASQUATCH/WALTER LAN,NAMORITA/NITA PRENTI,BALDER [ASGARDIAN],MEPHISTO,"CAGE, LUKE/CARL LUCA",KA-ZAR/KEVIN PLUNDER,BRUTE | MUTANT X-VER,"O'CONNELL, TARYN",MAINSPRING,"ASHER, MICHAEL",MISS THING/MARY,PLEX INTELLIGENCE,None
3,STORM/ORORO MUNROE S,VISION,DAREDEVIL/MATT MURDO,"NORRISS, SISTER BARB",INVISIBLE WOMAN/SUE,"HUDSON, HEATHER",JUSTICE II/VANCE AST,HOGUN [ASGARDIAN],MAKKARI/MIKE KHARY/I,IRON FIST/DANIEL RAN,ZABU,ICE-MAN | MUTANT X-V,WIDE LOAD ANNIE/ANNI,PSIREN,FAGIN,AMAZO-MAXI-WOMAN/,CAPTAIN GLORY,None
4,COLOSSUS II/PETER RA,HAWK,BLACK PANTHER/T'CHAL,"BANNER, BETTY ROSS T",SUB-MARINER/NAMOR MA,PUCK/EUGENE MILTON J,SPEEDBALL/ROBBIE BAL,FANDRAL [ASGARDIAN],THENA,HUMAN TORCH ANDROID/,"DE LA FONTAINE, CONT",WAR/ABRAHAM LINCOLN,POPPA WHEELER,PSIMON,HOFFMAN,"DARLEGUNG, GEN.",MERREE,None
5,PROFESSOR X/CHARLES,WASP/JANET VAN DYNE,"ROBERTSON, JOE",HELLCAT/PATSY WALKER,CRYSTAL [INHUMAN],GALACTUS/GALAN,MOON KNIGHT/MARC SPE,SIF,IKARIS/IKE HARRIS [E,"KNIGHT, MISTY","JONES, GABE","SUMMERS, SCOTT III |",RETREAD,SCENE STEALER,OSWALD,SWORDSMAN IV/,OUBLIETTE/,None
6,MARVEL GIRL/JEAN GRE,ANT-MAN/DR. HENRY J.,"PARKER, MAY","ROSS, GEN. THADDEUS","RICHARDS, FRANKLIN B",SHAMAN/MICHAEL TWOYO,NIGHT THRASHER/DUANE,THUNDERSTRIKE/ERIC K,HYPERION,BUCKY/BUCKY BARNES,DEATHLOK III/MICHAEL,FALLEN | MUTANT X-VE,RAZORBACK/,FIELDER,"ASHER, CARL",MANT/ERNEST,None,None
7,ANGEL/WARREN KENNETH,WONDER MAN/SIMON WIL,"LEEDS, BETTY BRANT",WONG,DR. DOOM/VICTOR VON,NORTHSTAR/JEAN-PAUL,GEE/ALEX POWER,VOLSTAGG,DEATH,"WING, COLLEEN","PIERCE, ALEXANDER GO",ELEKTRA | MUTANT X-V,"MCGRILL, MARY",KARMA 2013,"ASHER, DONNA",None,None,None
8,ICEMAN/ROBERT BOBBY,SHE-HULK/JENNIFER WA,"THOMPSON, EUGENE FLA",NIGHTHAWK II/KYLE RI,MEDUSA/MEDUSALITH AM,AURORA/JEANNE-MARIE,RAGE/ELVIN DARYL HAL,LOKI [ASGARDIAN],DR. SPECTRUM/JOSEPH,SHANG-CHI,"NEVILLE, KATE",CAPTAIN AMERICA | MU,SPRAGG,TANGERINE 2013,"NILES, SEN. CATHERIN",None,None,None
9,NIGHTCRAWLER/KURT WA,HERCULES [GREEK GOD],KINGPIN/WILSON FISK,DOC SAMSON/DR. LEONA,BLACK BOLT/BLACKANTO,STRONG GUY/GUIDO CAR,DAGGER/TANDY BOWEN,ENCHANTRESS/AMORA/HE,WHIZZER II/STANLEY S,WHIZZER/ROBERT L. FR,SLEEPWALKER,MAGNETO | MUTANT X-V,XEMNU,PSYLOCKE 2013,None,None,None,None


In [21]:
print(nx.community.modularity(subset_louvain.graph, subset_louvain.communities))

0.6593736886892159


In [22]:
len(subset_louvain.communities)

9

In [ ]:
popular = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed = False, cdn_resources='in_line')

popular.barnes_hut()
popular.from_nx(subset_contracted)

popular.show("popular.html")

In [32]:
popular_coms = pd.DataFrame(subset_louvain.communities)
popular_coms = popular_coms.transpose()

In [33]:
popular_coms.head(20)

,0,1,2,3,4,5,6,7,8
0,BLACK PANTHER/T'CHAL,WOLVERINE/LOGAN,SPIDER-MAN/PETER PAR,SHANG-CHI,MAN-THING/THEODORE T,"LUDLUM, ROSS",PANTHER CUB/,MISS AMERICA/MADELIN,STEEL SPIDER/OLLIE O
1,IRON MAN/TONY STARK,"WU, LEIKO",PUNISHER II/FRANK CA,STARSHINE II/BRANDY,VASHTI,OSWALD,MANT/ERNEST,PATRIOT/JEFF MACE,MASTER OF VENGEANCE
2,IRON MAN IV/JAMES R.,"RESTON, CLIVE","STACY, JILL","ROM, SPACEKNIGHT",GALACTUS/GALAN,HOFFMAN,SWORDSMAN IV/,DORMA [ATLANTEAN],None
3,"JACKSON, STEVE",MYSTIQUE/RAVEN DARKH,"WATSON-PARKER, MARY","CLARK, SARAH",SILVER SURFER/NORRIN,"NILES, SEN. CATHERIN","DARLEGUNG, GEN.",OLALLA,None
4,TORPEDO III/BROCK JO,BLOB/FRED J. DUKES,"LEEDS, BETTY BRANT","CLARK, JOHN",UATU,FAGIN,AMAZO-MAXI-WOMAN/,None,None
5,PYRO/ALLERDYCE JOHNN,ROGUE /,"WARD, SEN. STEWART",NOVA/RICHARD RIDER,INVISIBLE WOMAN/SUE,"ASHER, MICHAEL",MISS THING/MARY,None,None
6,AVALANCHE/DOMINIC PE,HYBRID/JAMES JIMMY M,"ROBERTSON, JOE",FIRESTAR/ANGELICA JO,THING/BENJAMIN J. GR,ORWELL,STERLING,None,None
7,DESTINY II/IRENE ADL,CYCLOPS/SCOTT SUMMER,"PARKER, MAY",NAMORITA/NITA PRENTI,DEATH,"ASHER, DONNA",None,None,None
8,SUB-MARINER/NAMOR MA,GHOST RIDER III/DAN,"THOMPSON, EUGENE FLA",SPEEDBALL/ROBBIE BAL,MR. FANTASTIC/REED R,"ASHER, CARL",None,None,None
9,THUNDERBALL/DR. ELIO,BEAST/HENRY &HANK& P,"JAMESON, J. JONAH",HULK/DR. ROBERT BRUC,ETERNITY/ADAM QADMON,None,None,None,None


In [ ]:
r = lambda: random.randint(0,255)

com_graph = nx.MultiGraph()
pos = nx.spring_layout(com_graph, scale = 500)

for community in coms.transpose().to_numpy():
    communityColorString = '#%02X%02X%02X' % (r(),r(),r())
    
    # Loop over each node 
    for node in list(community):
        if node == None: 
            continue
        else:
            com_graph.add_node(node, label = node, color=communityColorString)


com_graph.add_edges_from(random.sample(list(hero_edges.itertuples()), 5000))

['SUB-MARINER/NAMOR MA',
 'BEAST/HENRY &HANK& P',
 'DARK',
 'AURORA/JEANNE-MARIE ',
 'SHELLSHOCK/',
 'JAMESON, J. JONAH',
 'COOPER, DR. VALERIE',
 'SHATTERSTAR II/GAVEE',
 'EILIF DRAGONSLAYER',
 'CAPUTO, DR. JULIA',
 'CLARKE, GENERAL JERE',
 'HUMAN TORCH/JOHNNY S',
 'PHOBIUS',
 'MR. FANTASTIC/REED R',
 'BUTCH',
 'LANDERS, GENA',
 'THING/BENJAMIN J. GR',
 'INVISIBLE WOMAN/SUE ',
 'GRANT, GLORIA GLORY',
 'KARKAS [DEVIANT]',
 'DWORMAN, JIM',
 'SHATTERSTAR II/GAVEE',
 'STYGIAN STARBENDER',
 'JOSEPH',
 'BEAST/HENRY &HANK& P',
 'BLACK KNIGHT II/SIR ',
 'ROUGHHOUSE',
 'COUNTERWEIGHT II/KAT',
 'DRUID/DREDMUND CROMW',
 'MARVEL GIRL/JEAN GRE',
 'UATU',
 'KIRBY, JACK',
 'FRENCHIE/JEAN-PAUL D',
 'BEDLAM II/OLISA KABA',
 'WOLVERINE/LOGAN ',
 'BARNACLE/',
 'AVIA',
 'CADUCEUS',
 'SUB-MARINER/NAMOR MA',
 'FANDRAL [ASGARDIAN]',
 'ZARRKO, ARTHUR',
 'HUDSON, HEATHER',
 'HULK/DR. ROBERT BRUC',
 'LIVING TRIBUNAL',
 'MCLAREN, MEGAN',
 'AQUARIAN/WUNDARR',
 'URICH, BEN',
 'SYNCH/EVERETT THOMAS',
 "SUPER SKRUL

In [ ]:

net = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed = False, cdn_resources='in_line')

net.barnes_hut()
net.from_nx(com_graph)

net.show("Marvel2.html")

{'WOLVERINE/LOGAN ': 'WOLVERINE/LOGAN ', 'BEAST/HENRY &HANK& P': 'WOLVERINE/LOGAN ', 'CYCLOPS/SCOTT SUMMER': 'WOLVERINE/LOGAN ', 'STORM/ORORO MUNROE S': 'WOLVERINE/LOGAN ', 'COLOSSUS II/PETER RA': 'WOLVERINE/LOGAN ', 'PROFESSOR X/CHARLES ': 'WOLVERINE/LOGAN ', 'MARVEL GIRL/JEAN GRE': 'WOLVERINE/LOGAN ', 'ANGEL/WARREN KENNETH': 'WOLVERINE/LOGAN ', 'ICEMAN/ROBERT BOBBY ': 'WOLVERINE/LOGAN ', 'NIGHTCRAWLER/KURT WA': 'WOLVERINE/LOGAN ', 'ROGUE /': 'WOLVERINE/LOGAN ', 'SHADOWCAT/KATHERINE ': 'WOLVERINE/LOGAN ', 'CANNONBALL II/SAM GU': 'WOLVERINE/LOGAN ', 'PSYLOCKE/ELISABETH B': 'WOLVERINE/LOGAN ', 'WOLFSBANE/RAHNE SINC': 'WOLVERINE/LOGAN ', 'GAMBIT/REMY LEBEAU ': 'WOLVERINE/LOGAN ', 'MAGNETO/MAGNUS/ERIC ': 'WOLVERINE/LOGAN ', 'POLARIS/LORNA DANE': 'WOLVERINE/LOGAN ', 'SUMMERS, NATHAN CHRI': 'WOLVERINE/LOGAN ', 'SUNSPOT/ROBERTO DACO': 'WOLVERINE/LOGAN ', 'BANSHEE/SEAN CASSIDY': 'WOLVERINE/LOGAN ', 'BOOMER/TABITHA SMITH': 'WOLVERINE/LOGAN ', 'MACTAGGERT, MOIRA KI': 'WOLVERINE/LOGAN ', 'JUBILE

In [ ]:
# generate random number for color generation

#c = lambda: random.randint(0,255)

In [ ]:
graph_contracted.edges

EdgeView([('WOLVERINE/LOGAN ', 'CAPTAIN AMERICA'), ('WOLVERINE/LOGAN ', 'CAGE, LUKE/CARL LUCA'), ('WOLVERINE/LOGAN ', 'HULK/DR. ROBERT BRUC'), ('WOLVERINE/LOGAN ', 'QUASAR III/WENDELL V'), ('WOLVERINE/LOGAN ', 'THING/BENJAMIN J. GR'), ('WOLVERINE/LOGAN ', 'SPIDER-MAN/PETER PAR'), ('WOLVERINE/LOGAN ', 'THOR/DR. DONALD BLAK'), ('WOLVERINE/LOGAN ', 'HAVOK/ALEX SUMMERS '), ('WOLVERINE/LOGAN ', 'NOVA/RICHARD RIDER'), ('WOLVERINE/LOGAN ', 'PATRIOT/JEFF MACE'), ('WOLVERINE/LOGAN ', 'QABIRI'), ('WOLVERINE/LOGAN ', 'BAPTISTE, TANTE MATT'), ('WOLVERINE/LOGAN ', 'CHI-CHEE'), ('CAPTAIN AMERICA', 'HULK/DR. ROBERT BRUC'), ('CAPTAIN AMERICA', 'THOR/DR. DONALD BLAK'), ('CAPTAIN AMERICA', 'THING/BENJAMIN J. GR'), ('CAPTAIN AMERICA', 'NOVA/RICHARD RIDER'), ('CAPTAIN AMERICA', 'SPIDER-MAN/PETER PAR'), ('CAPTAIN AMERICA', 'QUASAR III/WENDELL V'), ('CAPTAIN AMERICA', 'CAGE, LUKE/CARL LUCA'), ('CAPTAIN AMERICA', 'PATRIOT/JEFF MACE'), ('CAPTAIN AMERICA', 'HAVOK/ALEX SUMMERS '), ('CAPTAIN AMERICA', 'CHI-CHEE'

In [ ]:
color_contracted = nx.Graph()

for node in graph_contracted.nodes:
    nodeColorString = '#%02X%02X%02X' % (c(),c(),c())
    color_contracted.add_node(node, label = node, color=nodeColorString)

color_contracted.add_edges_from(graph_contracted.edges)


In [ ]:
titleNet = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed = False, cdn_resources='in_line')

titleNet.barnes_hut()
titleNet.from_nx(color_contracted)

titleNet.show('colorCodedCompressed.html')

In [ ]:
graph_contracted.nodes
sample = graph_contracted.get_edge_data('SPIDER-MAN/PETER PAR','HULK/DR. ROBERT BRUC')
print(sample)

{'weight': 3417}


In [ ]:
# collapsed graph for visualization

nt = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed = False, cdn_resources='in_line')

nt.barnes_hut()
nt.from_nx(graph_contracted)

nt.show("Marvel_Super_Nodes.html")

In [ ]:
hg_sample = nx.from_pandas_edgelist(hero_edges.sample(5000), 'hero1', 'hero2')


net = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed = False)

net.barnes_hut(overlap=1)
net.from_nx(hg_sample)

net.show("Marvel.html")

In [ ]:
# try louvain again

second_run = louvain(MarvelCluster.graph)

In [261]:
len(second_run.communities)
nx.community.modularity(second_run.graph, second_run.communities)

0.47099584820390805